In [31]:
import pandas as pd
import numpy as np
import random
import scipy.io

#split dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

#accuracy,AUCROC,precision,recall,f1-score,AUCPR
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import average_precision_score

from sklearn.metrics import confusion_matrix

#gridsearch/randomsearch
from itertools import product
from tqdm import tqdm

#visualize results
import matplotlib.pyplot as plt

In [32]:
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import Subset,DataLoader,TensorDataset
from torchvision import datasets,transforms
import torch.nn.functional as F

In [33]:
def set_seed(seed):
  np.random.seed(seed)
  torch.manual_seed(seed)
  random.seed(seed)

  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

In [34]:
# import warnings
# warnings.filterwarnings("ignore")

if torch.cuda.is_available():
  is_cuda=True
  print('GPU is on')
else:
  is_cuda=False
  print('GPU is off')

GPU is on


In [35]:
def data_prepare(dataset,batch_size,target_num,contam_ratio,seed,hybrid): 
  #set seed for reproductive results
  set_seed(seed)

  if dataset == 'MNIST':
    transformation = transforms.Compose([transforms.Resize(img_size),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))])
    #entire training tensor
    train_tensor = torchvision.datasets.MNIST('data/',train=True,transform=transformation,download=True)
    #testing tensor
    test_tensor = torchvision.datasets.MNIST('data/',train=False,transform=transformation,download=True)

  elif dataset == 'FashionMNIST':
    transformation = transforms.Compose([transforms.Resize(img_size),
                        transforms.ToTensor(),
                        transforms.Normalize((0.5,), (0.5,))])
    #entire training tensor
    train_tensor = torchvision.datasets.FashionMNIST('data/',train=True,transform=transformation,download=True)
    #testing tensor
    test_tensor = torchvision.datasets.FashionMNIST('data/',train=False,transform=transformation,download=True)

  elif dataset == 'CIFAR10':
    transformation = transforms.Compose([transforms.Resize(img_size),
                    transforms.ToTensor(),
                    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))])
    #entire training tensor
    train_tensor = torchvision.datasets.CIFAR10('data/',train=True,transform=transformation,download=True)
    #testing tensor
    test_tensor = torchvision.datasets.CIFAR10('data/',train=False,transform=transformation,download=True)

  train_X = torch.empty([len(train_tensor),train_tensor[0][0].size(0),img_size,img_size])
  train_y = []

  if mode == 'leave_one_anomaly':
    #only using normal samples to train the model
    for i,data in enumerate(train_tensor):
      X,y = data
      train_X[i] = X
      if y != target_num:
        train_y.append(1)
      else:
        train_y.append(-1)

  elif mode == 'leave_one_normal':
    for i,data in enumerate(train_tensor):
      X,y = data
      train_X[i] = X
      if y == target_num:
        train_y.append(1)
      else:
        train_y.append(-1)
   
  train_y = np.array(train_y)

  index_contam = np.arange(len(train_y))[train_y == -1]
  index_contam = np.random.choice(index_contam,int(contam_ratio*len(index_contam)),replace = False)

  train_y[index_contam] = 1

  #normal training tensor and dataloader
  if not hybrid:
    index_subset = np.arange(len(train_y))[train_y == 1]
  else:
    index_subset = np.arange(len(train_y))

  train_tensor = TensorDataset(train_X,torch.tensor(train_y))
  train_tensor = Subset(train_tensor,index_subset)
  train_loader = torch.utils.data.DataLoader(train_tensor,batch_size = batch_size,shuffle = True,drop_last = True)

  ###########################################################################################################
  
  #testing label
  test_X = torch.empty([len(test_tensor),test_tensor[0][0].size(0),img_size,img_size])
  test_y = []

  for i,data in enumerate(test_tensor):
    X,y = data
    test_X[i] = X
    test_y.append(y)

  test_y = np.array(test_y)

  if mode == 'leave_one_anomaly':
    index_y_normal = np.arange(len(test_y))[test_y != target_num]
    index_y_anomaly = np.arange(len(test_y))[test_y == target_num]
  elif mode == 'leave_one_normal':
    index_y_normal = np.arange(len(test_y))[test_y == target_num]
    index_y_anomaly = np.arange(len(test_y))[test_y != target_num]  

      
  test_y_bin = test_y.copy()
  test_y_bin[index_y_normal] = 1
  test_y_bin[index_y_anomaly] = -1

  return train_loader,test_X,test_y_bin

In [36]:
class generator(nn.Module):
  def __init__(self,nc,ngf,nz,n_extra_layers=0):
    super(generator,self).__init__()
    
    self.encoder_1 = nn.Sequential(
        #(nc)*32*32
        nn.Conv2d(nc, ngf, 4, 2, 1, bias = False),
        nn.LeakyReLU(0.2, inplace = True),
        #ngf*16*16
        nn.Conv2d(ngf, ngf*2, 4, 2, 1, bias = False),
        nn.BatchNorm2d(ngf*2),
        nn.LeakyReLU(0.2, inplace = True),
        #(ngf*2)*8*8
        nn.Conv2d(ngf*2, ngf*4, 4, 2, 1, bias = False),
        nn.BatchNorm2d(ngf*4),
        nn.LeakyReLU(0.2, inplace = True),
        #(ngf*4)*4*4
        #final conv layer
        nn.Conv2d(ngf*4, nz, 4, 1, 0, bias = False)
    )
    
    self.decoder = nn.Sequential(
        nn.ConvTranspose2d(nz, ngf*4, 4, 1, 0, bias = False),
        nn.BatchNorm2d(ngf*4),
        nn.ReLU(inplace = True),
        #(ngf*4)*4*4
        nn.ConvTranspose2d(ngf*4, ngf*2, 4, 2, 1, bias = False),
        nn.BatchNorm2d(ngf*2),
        nn.ReLU(inplace = True),
        #(ngf*2)*8*8
        nn.ConvTranspose2d(ngf*2, ngf, 4, 2, 1, bias = False),
        nn.BatchNorm2d(ngf),
        nn.ReLU(inplace = True),
        #ngf*16*16
        nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias = False),
        nn.Tanh()
        #nc*32*32
    )

    self.encoder_2 = nn.Sequential(
        #(nc)*32*32
        nn.Conv2d(nc, ngf, 4, 2, 1, bias = False),
        nn.LeakyReLU(0.2, inplace = True),
        #ngf*16*16
        nn.Conv2d(ngf, ngf*2, 4, 2, 1, bias = False),
        nn.BatchNorm2d(ngf*2),
        nn.LeakyReLU(0.2, inplace = True),
        #(ngf*2)*8*8
        nn.Conv2d(ngf*2, ngf*4, 4, 2, 1, bias = False),
        nn.BatchNorm2d(ngf*4),
        nn.LeakyReLU(0.2, inplace = True),
        #(ngf*4)*4*4
        #final conv layer
        nn.Conv2d(ngf*4, nz, 4, 1, 0, bias = False)
    )
      
  def forward(self,X):
    z = self.encoder_1(X)
    X_hat = self.decoder(z)
    z_hat = self.encoder_2(X_hat)

    return z,X_hat,z_hat

In [37]:
class discriminator(nn.Module):
  def __init__(self,nc,ndf,nz):
    super(discriminator,self).__init__()
    
    self.feature = nn.Sequential(
        #(nc)*32*32
        nn.Conv2d(nc, ndf, 4, 2, 1, bias = False),
        nn.LeakyReLU(0.2, inplace = True),
        #ndf*16*16
        nn.Conv2d(ndf, ndf*2, 4, 2, 1, bias = False),
        nn.BatchNorm2d(ndf*2),
        nn.LeakyReLU(0.2, inplace = True),
        #(ndf*2)*8*8
        nn.Conv2d(ndf*2, ndf*4, 4, 2, 1, bias = False),
        nn.BatchNorm2d(ndf*4),
        nn.LeakyReLU(0.2, inplace = True),
        #(ndf*4)*4*4
        )

    self.classifier = nn.Sequential(
        #final conv layer
        nn.Conv2d(ndf*4, 1, 4, 1, 0, bias = False),
        nn.Sigmoid()
    )


    
  def forward(self,X):
    feature=self.feature(X)
    output=self.classifier(feature).view(-1,1).squeeze(1)

    return feature,output

In [38]:
def weights_init(mod):
  """
  Custom weights initialization called on netG, netD and netE
  :param m:
  :return:
  """
  classname = mod.__class__.__name__
  if classname.find('Conv') != -1:
      mod.weight.data.normal_(0.0, 0.02)
  elif classname.find('BatchNorm') != -1:
      mod.weight.data.normal_(1.0, 0.02)
      mod.bias.data.fill_(0)


# net_generator.apply(weights_init)
# net_discriminator.apply(weights_init)

In [39]:
def fit(dataloader,net_generator,net_discriminator,epochs,batch_size,print_loss):
  L1_criterion = nn.L1Loss(reduction='mean')
  L2_criterion = nn.MSELoss(reduction='mean')
  BCE_criterion = nn.BCELoss(reduction='mean')

  if is_cuda:
    L1_criterion.cuda()
    L2_criterion.cuda()
    BCE_criterion.cuda()

  loss_D_list = []
  loss_G_list = []

  for epoch in range(epochs):
    for i,data in enumerate(dataloader):
      X,_ = data
      y_real = torch.FloatTensor(batch_size).fill_(0)#real label=0,size=batch_size
      y_fake = torch.FloatTensor(batch_size).fill_(1)#fake label=1,size=batch_size

      if is_cuda:
        X = X.cuda()
        y_real = y_real.cuda()
        y_fake = y_fake.cuda()

      X = Variable(X)
      y_real = Variable(y_real)
      y_fake = Variable(y_fake)

      #zero grad for discriminator
      net_discriminator.zero_grad()
      #training the discriminator with real sample
      _,output = net_discriminator(X)
      loss_D_real = BCE_criterion(output,y_real)

      #training the discriminator with fake sample
      _,X_hat,_ = net_generator(X)
      _,output = net_discriminator(X_hat)

      loss_D_fake = BCE_criterion(output,y_fake)

      #entire loss in discriminator
      loss_D = (loss_D_real+loss_D_fake)/2
      
      loss_D.backward()
      optimizer_D.step()
      
      if loss_D<1e-5:
          net_discriminator.apply(weights_init)

      #training the generator based on the result from the discriminator
      net_generator.zero_grad()

      z,X_hat,z_hat = net_generator(X)

      #adversarial loss
      # _,output = net_discriminator(X_hat)
      # loss_G_adversarial = BCE_criterion(output,y_real)

      #latent loss
      feature_real,_ = net_discriminator(X)
      feature_fake,_ = net_discriminator(X_hat)

      loss_G_latent = L2_criterion(feature_fake,feature_real)
      
      #contexutal loss
      loss_G_contextual = L1_criterion(X,X_hat)
      #entire loss in generator

      #encoder loss
      loss_G_encoder = L2_criterion(z,z_hat)
      loss_G = 1*loss_G_latent + 50*loss_G_contextual + 1*loss_G_encoder
      #loss_G = 1*loss_G_adversarial + 50*loss_G_contextual + 1*loss_G_encoder

      loss_G.backward()
      optimizer_G.step()

      if i%50 == 0 & print_loss:
        print('[%d/%d] [%d/%d] Loss D: %.4f / Loss G: %.4f' % (epoch+1,epochs,i,len(dataloader),loss_D,loss_G))

      loss_D_list.append(loss_D)
      loss_G_list.append(loss_G)

In [40]:
def evaluation(test_X,model):
  z,X_hat,z_hat = model(test_X.cuda())
  #calculate the anomaly score in testing sample
  L2_criterion = nn.MSELoss(reduction='none')
  score = L2_criterion(z.cpu(),z_hat.cpu())

  #sum the score in each sample
  score = torch.sum(score,dim=1)
  score = score.view(-1).detach().numpy()

  return score

In [41]:
dataset = 'MNIST'
mode = 'leave_one_anomaly'

#anomaly number (leave one class out)
target_nums = [0,1,2,3,4,5,6,7,8,9]

#seed
seeds = [1,2,3,4,5]

#contaminating ratio
contam_ratios = [1.0,0.75,0.5,0.25,0.0]

In [42]:
#global parameters
img_size = 32
batch_size = 512 #default = 64
lr = 0.0002
beta1 = 0.5
epochs= 15 #default = 15

#size of latent vector
nz = 50 #default = 100

#filter size of generator
ngf = 32 #default = 64
#filter size of discriminator
ndf = 32 #default = 64

#output image channels
nc = 1

if dataset == 'CIFAR10':
  nc = 3

In [43]:
df_aucpr_mean = pd.DataFrame(data = None, index = target_nums, columns = contam_ratios)
df_aucpr_std = pd.DataFrame(data = None, index = target_nums, columns = contam_ratios)

In [ ]:
for ratio in contam_ratios:

  df_aucpr_local = pd.DataFrame(data = None, index = target_nums, columns = seeds)

  for num in target_nums:
    for seed in seeds:
      #data
      train_loader,test_X,test_y_bin = data_prepare(dataset = dataset, batch_size = batch_size, target_num = num, contam_ratio = ratio, seed = seed, hybrid = False)

      #model initializaiton
      set_seed(seed)
      net_generator = generator(nc = nc, ngf= ngf, nz = nz)
      net_discriminator = discriminator(nc = nc, ndf= ndf, nz = nz)

      if is_cuda:
        net_generator.cuda()
        net_discriminator.cuda()

      #weights initialization
      net_generator.apply(weights_init)
      net_discriminator.apply(weights_init)

      optimizer_G = torch.optim.Adam(net_generator.parameters(), lr, betas=(beta1, 0.999))
      optimizer_D = torch.optim.Adam(net_discriminator.parameters(), lr, betas=(beta1, 0.999))

      fit(dataloader = train_loader, 
        net_generator = net_generator, net_discriminator = net_discriminator,
        epochs = epochs, batch_size = batch_size, print_loss = True)
      
      #testing
      score = evaluation(test_X,net_generator)

      df_aucpr_local.loc[num,seed] = average_precision_score(y_true = test_y_bin, y_score = score, pos_label = -1)

  #store the result
  df_aucpr_mean.loc[:,ratio] = df_aucpr_local.apply(np.mean,axis=1)
  df_aucpr_std.loc[:,ratio] = df_aucpr_local.apply(np.std,axis=1)

df_aucpr_mean.to_csv('gan_aeplus_mean_' + dataset + '.csv',index = True)
df_aucpr_std.to_csv('gan_aeplus_std_' + dataset + '.csv',index = True)

[1/15] [0/117] Loss D: 0.6602 / Loss G: 33.8607
[1/15] [50/117] Loss D: 0.0046 / Loss G: 19.4437
[1/15] [100/117] Loss D: 0.0026 / Loss G: 16.0507
[2/15] [0/117] Loss D: 0.0024 / Loss G: 15.8818
[2/15] [50/117] Loss D: 0.0008 / Loss G: 14.5028
[2/15] [100/117] Loss D: 0.0006 / Loss G: 13.8074
[3/15] [0/117] Loss D: 0.0006 / Loss G: 13.7995
[3/15] [50/117] Loss D: 0.0004 / Loss G: 13.2305
[3/15] [100/117] Loss D: 0.0002 / Loss G: 12.7476
[4/15] [0/117] Loss D: 0.0002 / Loss G: 12.9460
[4/15] [50/117] Loss D: 0.0002 / Loss G: 12.7777
[4/15] [100/117] Loss D: 0.0002 / Loss G: 12.1628
[5/15] [0/117] Loss D: 0.0001 / Loss G: 12.2871
[5/15] [50/117] Loss D: 0.0001 / Loss G: 12.0702
[5/15] [100/117] Loss D: 0.0001 / Loss G: 11.8295
[6/15] [0/117] Loss D: 0.0001 / Loss G: 11.8495
[6/15] [50/117] Loss D: 0.0001 / Loss G: 11.7422
[6/15] [100/117] Loss D: 0.0001 / Loss G: 11.8258
[7/15] [0/117] Loss D: 0.0001 / Loss G: 11.8857
[7/15] [50/117] Loss D: 0.0001 / Loss G: 11.7097
[7/15] [100/117] Loss